# Raw Data Exploration - Marketing Analytics

This notebook explores the raw tables loaded into DuckDB that serve as the foundation for the **Marketing Analytics dbt project**. The goal is to understand the structure, data types, and content of each source table before building the staging and mart layers.

## Raw Tables

| Table | Description |
|-------|-------------|
| `raw.customers` | Customer profiles with demographics and signup info |
| `raw.campaigns` | Marketing campaigns across multiple channels |
| `raw.ad_clicks` | Individual ad click events with cost data |
| `raw.conversions` | Conversion events (purchases, signups, downloads) |

## Key Questions
- What columns are available and what are their data types?
- Are there any formatting inconsistencies that need cleaning in the staging layer?
- What transformations should be applied (casting, rounding, standardizing case)?

## Setup

Connect to the local DuckDB database that contains the raw marketing data.

In [22]:
import duckdb
import os

db_path = os.path.join(os.path.dirname(os.path.abspath('exploration.ipynb')), 'dev.duckdb')
con = duckdb.connect(db_path)
print(f"Connected to: {db_path} ✅")

Connected to: c:\Users\Brandon\OneDrive\Documentos\Brandon\2026\Proyectos personales\dbt\marketing_analytics\marketing_analytics\dev.duckdb ✅


## 1. Customers

Exploring `raw.customers` — contains customer profiles with names, email, country, and signup date.

**Observations:**
- `first_name` and `last_name` are separate fields — can be concatenated into `full_name`
- `email` has mixed casing — should be lowered in staging
- `signup_date` needs to be cast from timestamp to `DATE`

In [10]:
con.execute(

    """
    SELECT 
        customer_id, 
        first_name,
        last_name,
        first_name || ' ' || last_name as full_name, 
        lower(email) as email, 
        country, 
        cast(signup_date as date) as signup_date
    FROM raw.customers
    LIMIT 10
    """

).df()

,customer_id,first_name,last_name,full_name,email,country,signup_date
0,1,Danielle,Levy,Danielle Levy,ashleysellers@example.org,Sao Tome and Principe,2024-08-09
1,2,Angel,Ortega,Angel Ortega,gary31@example.com,Suriname,2024-04-16
2,3,Joshua,Lopez,Joshua Lopez,austingarcia@example.net,Iceland,2024-04-13
3,4,Jeffrey,Powers,Jeffrey Powers,courtney32@example.org,Saint Lucia,2026-01-16
4,5,Jill,Robinson,Jill Robinson,stokessuzanne@example.org,Tokelau,2025-03-27
5,6,Erica,Vance,Erica Vance,millerrachel@example.com,Saint Pierre and Miquelon,2025-09-14
6,7,Patricia,Ramirez,Patricia Ramirez,michaelcampbell@example.com,Cape Verde,2026-01-19
7,8,Christopher,Baird,Christopher Baird,nicholasbautista@example.com,Czech Republic,2025-06-01
8,9,Robert,Lee,Robert Lee,natalie15@example.org,Saint Helena,2025-08-23
9,10,Anthony,Parsons,Anthony Parsons,swalters@example.net,Seychelles,2024-12-08


## 2. Campaigns

Exploring `raw.campaigns` — marketing campaigns with channel, budget, status, and date range.

**Observations:**
- `budget_usd` has excessive decimal places — should be rounded to 2
- `status` values are lowercase strings: `active`, `paused`, `completed`
- `start_date` and `end_date` need casting to `DATE`
- `channel` values include: `paid_search`, `paid_social`, `email`, `display`

In [ ]:
con.execute(

    """
    SELECT 
        campaign_id, 
        campaign_name, 
        channel, 
        cast(start_date as date) start_date, 
        round(budget_usd, 2) as budget_usd, 
        lower(status) as status, 
        cast(end_date as date) end_date
    FROM raw.campaigns
    LIMIT 10
    """

).df()

,campaign_id,campaign_name,channel,start_date,budget_usd,status,end_date
0,1,Mesh Best-Of-Breed Supply-Chains,paid_search,2025-03-31,1884.37,completed,2025-04-21
1,2,Empower Vertical Content,paid_search,2025-11-03,12806.51,paused,2026-02-04
2,3,Synergize Robust Solutions,email,2025-04-01,4964.36,paused,2025-11-06
3,4,Extend Mission-Critical Users,paid_social,2025-07-02,18155.69,completed,2025-12-08
4,5,Implement Clicks-And-Mortar Platforms,paid_social,2025-07-13,17262.89,completed,2025-12-10
5,6,Mesh Open-Source Users,paid_social,2024-08-23,1881.72,active,2024-09-15
6,7,Scale 24/7 E-Commerce,paid_search,2024-11-05,5141.09,paused,2025-06-20
7,8,Strategize Viral Technologies,display,2024-04-18,13545.07,completed,2025-11-30
8,9,Unleash Distributed Users,paid_search,2024-10-10,4677.62,active,2025-07-28
9,10,Exploit Mission-Critical Eyeballs,display,2025-08-20,3080.08,paused,2025-10-31


## 3. Conversions

Exploring `raw.conversions` — conversion events tied to clicks, customers, and campaigns.

**Observations:**
- `conversion_type` values: `purchase`, `signup`, `download` — should be lowered for consistency
- `revenue_usd` has extra decimals — round to 2
- `conversion_date` needs casting to `DATE`
- `click_id` links back to `ad_clicks`, enabling click-to-conversion attribution

In [20]:
con.execute(

    """
    SELECT 
        conversion_id, 
        click_id, 
        customer_id, 
        campaign_id, 
        cast(conversion_date as date) as conversion_date, 
        round(revenue_usd, 2) revenue_usd, 
        lower(conversion_type) as conversion_type
    FROM raw.conversions
    LIMIT 10
    """

).df()

,conversion_id,click_id,customer_id,campaign_id,conversion_date,revenue_usd,conversion_type
0,1,33248,20106,166,2024-09-10,173.81,download
1,2,264887,14771,196,2024-05-17,442.09,signup
2,3,140110,2785,119,2024-10-11,302.25,purchase
3,4,127711,6380,158,2024-10-31,369.63,purchase
4,5,141145,36944,193,2025-06-16,465.70,signup
5,6,235406,34245,187,2025-02-21,170.37,signup
6,7,70045,36050,124,2025-11-30,212.48,download
7,8,232467,12866,77,2025-03-28,329.24,signup
8,9,202498,8239,135,2024-07-16,435.68,purchase
9,10,270633,20418,191,2024-11-21,497.03,signup


## 4. Ad Clicks

Exploring `raw.ad_clicks` — individual click events linking campaigns to customers.

**Observations:**
- `device` has mixed casing — should be lowered in staging (`mobile`, `desktop`, `tablet`)
- `cost_usd` has extra decimal places — round to 2
- `click_date` needs casting to `DATE`
- Each click ties a `campaign_id` to a `customer_id`, enabling join to both dimensions

In [24]:
con.execute(

    """
    SELECT 
      click_id, 
      campaign_id, 
      customer_id, 
      cast(click_date as date) as click_date, 
      lower(device) as device, 
      round(cost_usd, 2) as cost_usd
    FROM raw.ad_clicks
    LIMIT 10
    """

).df()

,click_id,campaign_id,customer_id,click_date,device,cost_usd
0,1,100,5997,2026-01-20,mobile,1.75
1,2,87,30085,2024-06-24,desktop,1.10
2,3,172,1846,2024-03-09,tablet,1.25
3,4,174,3805,2024-08-01,desktop,1.67
4,5,104,22140,2025-02-06,mobile,3.07
5,6,186,34962,2024-04-04,desktop,3.57
6,7,43,33496,2024-08-16,desktop,2.03
7,8,120,49396,2024-02-28,tablet,1.67
8,9,33,11436,2024-08-03,desktop,1.37
9,10,160,39669,2024-10-06,mobile,1.61


## Cleanup

In [25]:
con.close()
print("Connection closed ✅")

Connection closed ✅
